In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Pasul 1: Definirea modelelor Pydantic pentru ieșiri structurate

Aceste modele definesc **schema** pe care agenții o vor returna. Utilizarea `response_format` împreună cu Pydantic asigură:
- ✅ Extracție de date sigură din punct de vedere al tipurilor
- ✅ Validare automată
- ✅ Fără erori de analiză din răspunsuri în text liber
- ✅ Rutare condiționată ușoară bazată pe câmpuri


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Pasul 2: Crearea Instrumentului de Rezervare Hotelieră

Acest instrument este cel pe care **availability_agent** îl va apela pentru a verifica dacă există camere disponibile. Folosim decoratorul `@ai_function` pentru a:
- Transforma o funcție Python într-un instrument apelabil de AI
- Genera automat schema JSON pentru LLM
- Gestiona validarea parametrilor
- Permite invocarea automată de către agenți

Pentru acest demo:
- **Stockholm, Seattle, Tokyo, Londra, Amsterdam** → Au camere disponibile ✅
- **Toate celelalte orașe** → Fără camere disponibile ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Pasul 3: Definirea funcțiilor de condiție pentru rutare

Aceste funcții analizează răspunsul agentului și determină ce cale să urmeze în fluxul de lucru.

**Model cheie:**
1. Verifică dacă mesajul este `AgentExecutorResponse`
2. Analizează ieșirea structurată (modelul Pydantic)
3. Returnează `True` sau `False` pentru a controla rutarea

Fluxul de lucru va evalua aceste condiții pe **legături** pentru a decide ce executor să invoce în continuare.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Pasul 4: Crearea unui Executor Personalizat pentru Afișare

Executorii sunt componente ale fluxului de lucru care efectuează transformări sau efecte secundare. Folosim decoratorul `@executor` pentru a crea un executor personalizat care afișează rezultatul final.

**Concepte Cheie:**
- `@executor(id="...")` - Înregistrează o funcție ca executor al fluxului de lucru
- `WorkflowContext[Never, str]` - Indicii de tip pentru intrare/ieșire
- `ctx.yield_output(...)` - Returnează rezultatul final al fluxului de lucru


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Pasul 5: Încarcă Variabilele de Mediu

Configurează clientul LLM. Acest exemplu funcționează cu:
- **Modele GitHub** (Nivel gratuit cu token GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Pasul 6: Crearea agenților AI cu rezultate structurate

Creăm **trei agenți specializați**, fiecare învelit într-un `AgentExecutor`:

1. **availability_agent** - Verifică disponibilitatea hotelurilor folosind instrumentul
2. **alternative_agent** - Sugerează orașe alternative (când nu sunt camere disponibile)
3. **booking_agent** - Încurajează rezervarea (când camerele sunt disponibile)

**Caracteristici cheie:**
- `tools=[hotel_booking]` - Oferă instrumentul agentului
- `response_format=PydanticModel` - Obligă generarea unui output JSON structurat
- `AgentExecutor(..., id="...")` - Învelește agentul pentru utilizare în fluxul de lucru


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Pasul 7: Construirea fluxului de lucru cu ramuri condiționale

Acum folosim `WorkflowBuilder` pentru a construi graful cu rutare condițională:

**Structura fluxului de lucru:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Metode cheie:**
- `.set_start_executor(...)` - Setează punctul de intrare
- `.add_edge(from, to, condition=...)` - Adaugă o ramură condițională
- `.build()` - Finalizează fluxul de lucru


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Pasul 8: Rulează Cazul de Test 1 - Oraș FĂRĂ Disponibilitate (Paris)

Să testăm scenariul de **fără disponibilitate** cerând hoteluri în Paris (care nu are camere în simularea noastră).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Pasul 9: Rulează Cazul de Test 2 - Oraș CU Disponibilitate (Stockholm)

Acum să testăm calea **disponibilității** solicitând hoteluri în Stockholm (care are camere în simularea noastră).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Concluzii și Pași Următori

### ✅ Ce Ai Învățat:

1. **Modelul WorkflowBuilder**
   - Folosește `.set_start_executor()` pentru a defini punctul de intrare
   - Folosește `.add_edge(from, to, condition=...)` pentru rutare condiționată
   - Apelează `.build()` pentru a finaliza fluxul de lucru

2. **Rutare Condiționată**
   - Funcțiile de condiție inspectează `AgentExecutorResponse`
   - Analizează ieșirile structurate pentru a lua decizii de rutare
   - Returnează `True` pentru a activa o legătură, `False` pentru a o sări

3. **Integrarea Instrumentelor**
   - Folosește `@ai_function` pentru a transforma funcțiile Python în instrumente AI
   - Agenții apelează automat instrumentele atunci când este necesar
   - Instrumentele returnează JSON pe care agenții îl pot analiza

4. **Ieșiri Structurate**
   - Folosește modele Pydantic pentru extragerea sigură a datelor
   - Setează `response_format=MyModel` când creezi agenți
   - Analizează răspunsurile cu `Model.model_validate_json()`

5. **Executori Personalizați**
   - Folosește `@executor(id="...")` pentru a crea componente ale fluxului de lucru
   - Executorii pot transforma date sau efectua acțiuni secundare
   - Folosește `ctx.yield_output()` pentru a produce rezultate ale fluxului de lucru

### 🚀 Aplicații în Lumea Reală:

- **Rezervări de Călătorii**: Verifică disponibilitatea, sugerează alternative, compară opțiuni
- **Serviciu Clienți**: Rutează în funcție de tipul problemei, sentiment, prioritate
- **E-commerce**: Verifică inventarul, sugerează alternative, procesează comenzi
- **Moderare de Conținut**: Rutează în funcție de scorurile de toxicitate, semnalările utilizatorilor
- **Fluxuri de Aprobare**: Rutează în funcție de sumă, rolul utilizatorului, nivelul de risc
- **Procesare Multi-etapă**: Rutează în funcție de calitatea datelor, completitudine

### 📚 Pași Următori:

- Adaugă condiții mai complexe (criterii multiple)
- Implementează bucle cu gestionarea stării fluxului de lucru
- Adaugă sub-fluxuri pentru componente reutilizabile
- Integrează cu API-uri reale (rezervări hoteluri, sisteme de inventar)
- Adaugă gestionarea erorilor și căi de rezervă
- Vizualizează fluxurile de lucru cu instrumentele de vizualizare integrate



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
